# Exercises

## Take the work we did in the lessons further:

In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report as class_rep

from env import get_connection
from adam_prepare import clean, lemmatize

In [3]:
url = get_connection('spam_db')

query = 'SELECT * FROM spam'

df = pd.read_sql(query, url, index_col='id')

df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['clean_text'] = df.text.apply(clean, args=['us'])
df.head()

,label,text,clean_text
id,,,
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah ' think goes usf lives around though


In [5]:
df['lemmas'] = df.clean_text.apply(lemmatize)
df.head()

,label,text,clean_text,lemmas
id,,,,
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif oni,ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah ' think goes usf lives around though,nah ' think go usf life around though


In [6]:
X = df.lemmas
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=42)

In [7]:
tfidf = TfidfVectorizer()

X_train_tfidfs = tfidf.fit_transform(X_train)
X_test_tfidfs = tfidf.transform(X_test)

X_train_tfidfs[:10]

<10x7197 sparse matrix of type '<class 'numpy.float64'>'
	with 63 stored elements in Compressed Sparse Row format>

In [8]:
sh_df= pd.DataFrame(X_train_tfidfs.todense(), columns=tfidf.get_feature_names_out())
sh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Columns: 7197 entries, 008704050406 to zouk
dtypes: float64(7197)
memory usage: 214.1 MB


In [9]:
sh_df.describe().T

,count,mean,std,min,25%,50%,75%,max
008704050406,3900.0,0.000140,0.006192,0.0,0.0,0.0,0.0,0.279046
0089my,3900.0,0.000066,0.004118,0.0,0.0,0.0,0.0,0.257146
0121,3900.0,0.000092,0.005755,0.0,0.0,0.0,0.0,0.359427
0125698789,3900.0,0.000144,0.008995,0.0,0.0,0.0,0.0,0.561714
02,3900.0,0.000086,0.005343,0.0,0.0,0.0,0.0,0.333699
...,...,...,...,...,...,...,...,...
zed,3900.0,0.000324,0.008303,0.0,0.0,0.0,0.0,0.234107
zero,3900.0,0.000105,0.006562,0.0,0.0,0.0,0.0,0.409784
zogtorius,3900.0,0.000097,0.006035,0.0,0.0,0.0,0.0,0.376909
zoom,3900.0,0.000076,0.004770,0.0,0.0,0.0,0.0,0.297889


In [10]:
sh_df

,008704050406,0089my,0121,0125698789,02,020603,0207,02070836089,02072069400,02073162414,...,yupz,ywhere,zaher,zealand,zebra,zed,zero,zogtorius,zoom,zouk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
lm = LogisticRegression()

lm.fit(X_train_tfidfs, y_train)

LogisticRegression()

In [12]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': lm.predict(X_train_tfidfs)})
y_train_res.head()

,actual,preds
id,,
708,spam,spam
4338,ham,ham
5029,ham,ham
4921,ham,ham
2592,ham,ham


In [13]:
print(class_rep(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      3372
        spam       0.99      0.74      0.85       528

    accuracy                           0.96      3900
   macro avg       0.98      0.87      0.91      3900
weighted avg       0.96      0.96      0.96      3900



In [14]:
y_test_res = pd.DataFrame({'actual': y_test,
                           'preds': lm.predict(X_test_tfidfs)})
y_test_res.head()

,actual,preds
id,,
3245,ham,ham
944,ham,ham
1044,spam,ham
2484,ham,ham
812,spam,spam


In [15]:
print(class_rep(y_test_res.actual, y_test_res.preds))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97      1453
        spam       0.96      0.64      0.77       219

    accuracy                           0.95      1672
   macro avg       0.95      0.82      0.87      1672
weighted avg       0.95      0.95      0.94      1672



-------

### 1. What other types of models (i.e. different classifcation algorithms) could you use?

## **(Lesson Spam and Ham text data)**

### - ***K-Nearest Neighbors***

In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train_tfidfs, y_train)

KNeighborsClassifier()

In [17]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': knn.predict(X_train_tfidfs)})
y_train_res.head()

,actual,preds
id,,
708,spam,ham
4338,ham,ham
5029,ham,ham
4921,ham,ham
2592,ham,ham


In [18]:
print(class_rep(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95      3372
        spam       1.00      0.37      0.54       528

    accuracy                           0.91      3900
   macro avg       0.95      0.68      0.75      3900
weighted avg       0.92      0.91      0.90      3900



In [19]:
y_test_res = pd.DataFrame({'actual': y_test,
                           'preds': knn.predict(X_test_tfidfs)})
y_test_res.head()

,actual,preds
id,,
3245,ham,ham
944,ham,ham
1044,spam,ham
2484,ham,ham
812,spam,spam


In [20]:
print(class_rep(y_test_res.actual, y_test_res.preds))

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95      1453
        spam       1.00      0.31      0.47       219

    accuracy                           0.91      1672
   macro avg       0.95      0.66      0.71      1672
weighted avg       0.92      0.91      0.89      1672



  ----------------

### - ***Random Forest Classifier***

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train_tfidfs, y_train)

RandomForestClassifier()

In [22]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': rf.predict(X_train_tfidfs)})
y_train_res.head()

,actual,preds
id,,
708,spam,spam
4338,ham,ham
5029,ham,ham
4921,ham,ham
2592,ham,ham


In [23]:
print(class_rep(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00      3372
        spam       1.00      1.00      1.00       528

    accuracy                           1.00      3900
   macro avg       1.00      1.00      1.00      3900
weighted avg       1.00      1.00      1.00      3900



In [24]:
y_test_res = pd.DataFrame({'actual': y_test,
                           'preds': rf.predict(X_test_tfidfs)})
y_test_res.head()

,actual,preds
id,,
3245,ham,ham
944,ham,ham
1044,spam,spam
2484,ham,ham
812,spam,spam


In [25]:
print(class_rep(y_test_res.actual, y_test_res.preds))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1453
        spam       0.99      0.82      0.90       219

    accuracy                           0.98      1672
   macro avg       0.98      0.91      0.94      1672
weighted avg       0.98      0.98      0.98      1672



---------------  

## **(codeup text data)**

In [26]:
import acquire as a
from prepare import clean, prep_text_data, lemmatize 

In [27]:
codeup_df = a.get_blog_articles_data()
codeup_df.head(1)

,title,content
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...


In [29]:
codeup_df['clean_text'] = codeup_df.content.apply(clean, args=['us'])
codeup_df['lemmas'] = codeup_df.clean_text.apply(lemmatize)
codeup_df

,title,content,clean_text,lemmas
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,ay traditionally known sian merican acific sla...,ay traditionally known sian merican acific sla...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,omen tech anelist potlight agdalena ahn odeup ...,omen tech anelist potlight agdalena ahn odeup ...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...,omen tech anelist potlight achel obbinsayhill ...,omen tech anelist potlight achel obbinsayhill ...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight – Sarah Mell...,omen tech anelist potlight arah ellor odeup ho...,omen tech anelist potlight arah ellor odeup ho...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight – Madeleine ...,omen tech anelist potlight adeleine apper odeu...,omen tech anelist potlight adeleine apper odeu...
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight –...,lack excellence tech anelist potlight ilmarie ...,lack excellence tech anelist potlight ilmarie ...


In [28]:
codeup_df_ = a.get_blog_articles_data()
codeup_df_.rename(columns={'content': 'original'}, inplace=True)
prep_text_data(codeup_df_, 'original', extra_words = ['ha'], exclude_words = ['no']).head()

,title,original,clean,stemmed,lemmatized
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,ay traditionally known sian merican acific sla...,ay tradit known sian merican acif slander erit...,ay traditionally known sian merican acific sla...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,omen tech anelist potlight agdalena ahn odeup ...,omen tech anelist potlight agdalena ahn odeup ...,omen tech anelist potlight agdalena ahn odeup ...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...,omen tech anelist potlight achel obbinsayhill ...,omen tech anelist potlight achel obbinsayhil o...,omen tech anelist potlight achel obbinsayhill ...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight – Sarah Mell...,omen tech anelist potlight arah ellor odeup ho...,omen tech anelist potlight arah ellor odeup ho...,omen tech anelist potlight arah ellor odeup ho...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight – Madeleine ...,omen tech anelist potlight adeleine apper odeu...,omen tech anelist potlight adelein apper odeup...,omen tech anelist potlight adeleine apper odeu...


In [30]:
X = codeup_df.lemmas
y = codeup_df.title

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=42)

In [31]:
tfidfs = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf[:4]

<4x317 sparse matrix of type '<class 'numpy.float64'>'
	with 459 stored elements in Compressed Sparse Row format>

In [32]:
baseline_acc = y_train.value_counts().max() / y_train.shape[0] * 100

print(f'baseline accuracy: {round(baseline_acc, 2)} \n\n...sheeeeeee')

baseline accuracy: 25.0 

...sheeeeeee


In [33]:
pd.DataFrame(X_train_tfidf.todense(), columns=tfidf.get_feature_names_out())

,2019,2022,2023,22,22nd,29th,able,absolutely,achel,achels,...,went,within,woman,work,worked,working,xcellence,year,yracuse,ystems
0,0.000000,0.000000,0.051973,0.099596,0.099596,0.000000,0.078523,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.099596,0.000000,0.000000,0.000000
1,0.000000,0.102149,0.053306,0.000000,0.000000,0.065200,0.040268,0.000000,0.459671,0.102149,...,0.000000,0.051075,0.032600,0.000000,0.000000,0.040268,0.000000,0.000000,0.051075,0.051075
2,0.083564,0.000000,0.043607,0.000000,0.000000,0.106675,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.053338,0.065883,0.065883,0.065883,0.000000,0.065883,0.000000,0.000000
3,0.000000,0.000000,0.045891,0.000000,0.000000,0.112264,0.000000,0.087941,0.000000,0.000000,...,0.087941,0.000000,0.056132,0.208002,0.069334,0.000000,0.000000,0.138668,0.000000,0.000000


In [34]:
lm = LogisticRegression()

lm.fit(X_train_tfidf, y_train)

LogisticRegression()

In [35]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': lm.predict(X_train_tfidf)})
y_train_res.head()

,actual,preds
5,Black Excellence in Tech: Panelist Spotlight –...,Black Excellence in Tech: Panelist Spotlight –...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in Tech: Panelist Spotlight – Madeleine ...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in Tech: Panelist Spotlight – Sarah Mellor


In [36]:
print(class_rep(y_train_res.actual, y_train_res.preds))

                                                                          precision    recall  f1-score   support

Black Excellence in Tech: Panelist Spotlight – Wilmarie De La Cruz Mejia       1.00      1.00      1.00         1
                    Women in Tech: Panelist Spotlight – Madeleine Capper       1.00      1.00      1.00         1
                        Women in Tech: Panelist Spotlight – Sarah Mellor       1.00      1.00      1.00         1
              Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill       1.00      1.00      1.00         1

                                                                accuracy                           1.00         4
                                                               macro avg       1.00      1.00      1.00         4
                                                            weighted avg       1.00      1.00      1.00         4



In [37]:
y_test_res = pd.DataFrame({'actual': y_test,
                           'preds': lm.predict(X_test_tfidf)})
y_test_res.head()

,actual,preds
0,Spotlight on APIDA Voices: Celebrating Heritag...,Women in tech: Panelist Spotlight – Rachel Rob...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in Tech: Panelist Spotlight – Madeleine ...


In [38]:
print(class_rep(y_test_res.actual, y_test_res.preds))

                                                                                        precision    recall  f1-score   support

Spotlight on APIDA Voices: Celebrating Heritage and Inspiring Change ft. Arbeena Thapa       0.00      0.00      0.00       1.0
                                  Women in Tech: Panelist Spotlight – Madeleine Capper       0.00      0.00      0.00       0.0
                                    Women in tech: Panelist Spotlight – Magdalena Rahn       0.00      0.00      0.00       1.0
                            Women in tech: Panelist Spotlight – Rachel Robbins-Mayhill       0.00      0.00      0.00       0.0

                                                                              accuracy                           0.00       2.0
                                                                             macro avg       0.00      0.00      0.00       2.0
                                                                          weighted avg       0.00     

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/pytho

### 2. How do the models compare when trained on term frequency data alone, instead of TF-IDF values?

In [40]:
url = get_connection('spam_db')

query = 'SELECT * FROM spam'

df = pd.read_sql(query, url, index_col='id')

df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [41]:
df['clean_text'] = df.text.apply(clean, args=['us'])
df.head()

,label,text,clean_text
id,,,
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy vailable bugis n great worl...
1,ham,Ok lar... Joking wif u oni...,k lar oking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,ree entry 2 wkly comp win final tkts 21st ay 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",ah dont think goes usf lives around though


In [42]:
df['lemmas'] = df.clean_text.apply(lemmatize)
df.head()

,label,text,clean_text,lemmas
id,,,,
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy vailable bugis n great worl...,jurong point crazy vailable bugis n great worl...
1,ham,Ok lar... Joking wif u oni...,k lar oking wif oni,k lar oking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,ree entry 2 wkly comp win final tkts 21st ay 2...,ree entry 2 wkly comp win final tkts 21st ay 2...
3,ham,U dun say so early hor... U c already then say...,dun say early hor c already say,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",ah dont think goes usf lives around though,ah dont think go usf life around though


In [43]:
X = df.lemmas
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=42)

In [55]:
tfidf = TfidfVectorizer(use_idf=False) # used an aurgument found in the sklearn documentation

X_train_tfidfs = tfidf.fit_transform(X_train)
X_test_tfidfs = tfidf.transform(X_test)

X_train_tfidfs[:10]

<10x7351 sparse matrix of type '<class 'numpy.float64'>'
	with 64 stored elements in Compressed Sparse Row format>

In [45]:
sh_df= pd.DataFrame(X_train_tfidfs.todense(), columns=tfidf.get_feature_names_out())
sh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Columns: 7351 entries, 008704050406 to zzit
dtypes: float64(7351)
memory usage: 218.7 MB


In [46]:
sh_df.describe().T

,count,mean,std,min,25%,50%,75%,max
008704050406,3900.0,0.000123,0.005415,0.0,0.0,0.0,0.0,0.242536
0089my,3900.0,0.000050,0.003140,0.0,0.0,0.0,0.0,0.196116
0121,3900.0,0.000077,0.004828,0.0,0.0,0.0,0.0,0.301511
0125698789,3900.0,0.000256,0.016013,0.0,0.0,0.0,0.0,1.000000
01327,3900.0,0.000121,0.005337,0.0,0.0,0.0,0.0,0.235702
...,...,...,...,...,...,...,...,...
zealand,3900.0,0.000037,0.002311,0.0,0.0,0.0,0.0,0.144338
zed,3900.0,0.000319,0.008196,0.0,0.0,0.0,0.0,0.242536
zero,3900.0,0.000074,0.004623,0.0,0.0,0.0,0.0,0.288675
zoom,3900.0,0.000052,0.003269,0.0,0.0,0.0,0.0,0.204124


In [10]:
sh_df

,008704050406,0089my,0121,0125698789,02,020603,0207,02070836089,02072069400,02073162414,...,yupz,ywhere,zaher,zealand,zebra,zed,zero,zogtorius,zoom,zouk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
lm = LogisticRegression()

lm.fit(X_train_tfidfs, y_train)

LogisticRegression()

In [48]:
y_train_res = pd.DataFrame({'actual': y_train,
                            'preds': lm.predict(X_train_tfidfs)})
y_train_res.head()

,actual,preds
id,,
708,spam,spam
4338,ham,ham
5029,ham,ham
4921,ham,ham
2592,ham,ham


In [49]:
print(class_rep(y_train_res.actual, y_train_res.preds))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.98      3372
        spam       1.00      0.70      0.82       528

    accuracy                           0.96      3900
   macro avg       0.98      0.85      0.90      3900
weighted avg       0.96      0.96      0.96      3900



In [50]:
y_test_res = pd.DataFrame({'actual': y_test,
                           'preds': lm.predict(X_test_tfidfs)})
y_test_res.head()

,actual,preds
id,,
3245,ham,ham
944,ham,ham
1044,spam,ham
2484,ham,ham
812,spam,spam


In [51]:
print(class_rep(y_test_res.actual, y_test_res.preds))

              precision    recall  f1-score   support

         ham       0.94      1.00      0.97      1453
        spam       0.99      0.59      0.74       219

    accuracy                           0.95      1672
   macro avg       0.97      0.80      0.86      1672
weighted avg       0.95      0.95      0.94      1672



In [54]:
#TF_IDF

#               precision    recall  f1-score   support

#          ham       0.95      1.00      0.97      1453
#         spam       0.96      0.64      0.77       219

#     accuracy                           0.95      1672
#    macro avg       0.95      0.82      0.87      1672
# weighted avg       0.95      0.95      0.94      1672

### They perform almost the same and a little better when looking at precision, but not ehrm comparing f1 score and recall